In [31]:
from pathlib import Path
import pandas as pd
from PIL import Image
from matplotlib.figure import Figure
import xlwings as xw
# Requires a license key: https://www.xlwings.org/trial
from xlwings.pro import Markdown, MarkdownStyle
from xlwings.pro.reports import create_report

### Some data

In [32]:
# Matplotlib
fig = Figure(figsize=(4, 3))
ax = fig.add_subplot(111)
ax.plot([1, 2, 3, 4, 5])

# Pandas DataFrame
perf_data = pd.DataFrame(index=['South', 'North', 'West'],
                         columns=[2020, 2021],
                         data=[[1., 2.], [3., 4.], [5, 6]])
perf_data.index.name = 'Sales'
# Picture
logo = Image.open(Path('xlwings.jpg'))

# Float
perf = 0.12 * 100

In [33]:
# Markdown

mytext = f"""\
# Q1 2021 Results

The perfomance was {perf}%.
This was due to the following points:

* More sales
* Cost cuts

# Sales were strong

*Automation was the most important driver*.
More info on request.
"""

style = MarkdownStyle()
style.h1.font.color = (21, 164, 58)
style.h1.font.size = 14

### Render a template on a sheet (new in 0.22.0)

In [34]:
wb = xw.Book('report_template.xlsx')
template_sheet = wb.sheets[0]
# Copy the template sheet first!
# You could provide a name via copy(name=...)
report_sheet = template_sheet.copy()
# Collect all template vars (optional)
data = dict(perf_data=perf_data, logo=logo,
            perf=perf, fig=fig, summary=Markdown(mytext, style))
report_sheet.render_template(**data)

report_sheet['A1'].select()
logo.close()

### PDF report

In [30]:
report_sheet.to_pdf()

### create_report is a convenience wrapper to handle a whole Book

In [35]:
wb = create_report('report_template.xlsx',
                   'report.xlsx',
                   **data)

In [36]:
wb.close()

### Using an invisible App

In [38]:
app = xw.App(visible=False)
wb = create_report('report_template.xlsx',
                   'report.xlsx',
                   **data,
                   app=app,
                   book_settings={'update_links': False})
wb.sheets['Sheet1'].to_pdf()

# Clean up
wb.sheets.active['A1'].select()
wb.save()
wb.close()
app.quit()